In [ ]:
import os
import cv2
import time
import scipy as sp
import numpy as np
import random as rn
import pandas as pd
from tqdm import tqdm
from PIL import Image
from functools import partial
import matplotlib.pyplot as plt

# Machine Learning
import tensorflow as tf
import keras
from keras import initializers
from keras import regularizers
from keras import constraints
from keras import backend as K
from keras.activations import elu
from keras.optimizers import Adam
from keras.models import Sequential
from keras.engine import Layer, InputSpec
from keras.utils.generic_utils import get_custom_objects
from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau
from keras.layers import Dense, Conv2D, Flatten, GlobalAveragePooling2D, Dropout,MaxPooling2D,BatchNormalization,GlobalMaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import cohen_kappa_score

# Specify title of our final model
SAVED_MODEL_NAME = 'effnet_modelB5.h5'

# Set seed for reproducability
seed = 1234
rn.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)

# For keeping time. GPU limit for this competition is set to ± 9 hours.
t_start = time.time()

In [ ]:

from tensorflow.keras.models import load_model

In [ ]:


from keras.applications.xception import Xception
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np



In [ ]:


effnet = tf.keras.applications.MobileNetV2(input_shape=(256,256,3),
                                               include_top=False,
                                               weights='imagenet')


In [ ]:
for layer in effnet.layers:
    layer.trainable = True

In [ ]:

metrics=tf.keras.metrics.AUC(name='auc')
def build_model():

    model = Sequential()
    model.add(effnet)
    model.add(GlobalAveragePooling2D())
    model.add(Dropout(0.25))
#    model.add(Dense(2, activation=elu))
    model.add(Dense(1, activation="sigmoid"))
    model.compile(loss='binary_crossentropy',
                  optimizer=keras.optimizers.Adam(learning_rate=0.00005, beta_1=0.9, beta_2=0.999, amsgrad=False),
                  metrics=metrics)
    print(model.summary())
    return model

# Initialize model
model = build_model()



In [ ]:
import pandas as pd
df=pd.read_csv("../input/jpeg-melanoma-256x256/train.csv")

In [ ]:
from sklearn.model_selection import train_test_split

train, valid = train_test_split(df, test_size=0.2)

In [ ]:
train

In [ ]:
train["target"]=train["target"].astype(str)
valid["target"]=valid["target"].astype(str)

In [ ]:
train["image_name"]=train["image_name"]+".jpg"
valid["image_name"]=valid["image_name"]+".jpg"

In [ ]:
train

In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1/255,
        zoom_range=0.0,
        horizontal_flip=True)


test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_dataframe(
        dataframe=train,
        directory='../input/jpeg-melanoma-256x256/train/',
        x_col="image_name",
        y_col="target",
        target_size=(256, 256),
        batch_size=32,
        class_mode='binary')

validation_generator = test_datagen.flow_from_dataframe(
        dataframe=valid,
        directory='../input/jpeg-melanoma-256x256/train/',
        x_col="image_name",
        y_col="target",
        target_size=(256, 256),
        batch_size=16,
        class_mode='binary')

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='auto', verbose=1, patience=4)
rlr =ReduceLROnPlateau(monitor='val_loss', 
                        factor=0.5, 
                        patience=3, 
                        verbose=1, 
                        mode='auto', 
                        epsilon=0.0001)


#class_weight = {0: 1.99,
#               1:1.}

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=SAVED_MODEL_NAME,
    save_weights_only=True,
    monitor='val_acc',
    mode='max',
    save_best_only=True)
history = model.fit(
        train_generator,
        epochs=5,
        shuffle=True,
        validation_data=validation_generator,
        callbacks=[model_checkpoint_callback])



In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model,to_file='model_plot.png',show_shapes=True,show_layer_names=True)

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

auc = history.history['auc']
val_auc = history.history['val_auc']

loss = history.history['loss']
val_loss = history.history['val_loss']
epochs=5
epochs_range = range(epochs)

plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.plot(epochs_range, auc, label='Training Accuracy')
plt.plot(epochs_range, val_auc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
plt.ylabel("Accuracy (training and validation)")
plt.xlabel("Training Steps")

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.ylabel("Loss (training and validation)")
plt.xlabel("Training Steps")
plt.show()

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
predictions1=model.predict(validation_generator,steps=len(validation_generator))
y=np.argmax(predictions1,axis=1)

print('Classification Report')
cr=classification_report(y_true=validation_generator.classes,y_pred=y,target_names=validation_generator.class_indices)
print(cr)
evaluates=model.evaluate(validation_generator)
print(evaluates)

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sn
print('Confusion Matrix')
cm = confusion_matrix(validation_generator.classes,y)
df = pd.DataFrame(cm,columns=validation_generator.class_indices)
plt.figure(figsize=(10,7))
sn.heatmap(df,annot=True)